In [17]:
import torch
import numpy as np
from PIL import Image, ImageSequence

from algorithms.diffusion_wrapper import DiffusionWrapper
from algorithms.heatmap_generator import HeatmapGenerator
from algorithms.zero_shot_tracker import ZeroShotTracker


diffusion_wrapper = DiffusionWrapper('../text-to-video-ms-1.7b/')
heatmap_generator = HeatmapGenerator()
zero_shot_tracker = ZeroShotTracker()

Loading pipeline components...: 100%|██████████| 5/5 [00:00<00:00,  7.55it/s]


In [23]:
from evaluation.evaluation_datasets import create_davis_dataset

davis_dataset = create_davis_dataset('../tapvid_davis/tapvid_davis.pkl')

first_video = {}

for data in davis_dataset:
    first_video = data['davis']
    break

video_tensor = torch.tensor(first_video['video'])

{'davis': {'video': array([[[[[120, 108,  96],
          [131, 121, 106],
          [ 99,  91,  74],
          ...,
          [122, 107, 100],
          [136, 121, 117],
          [109,  94,  91]],

         [[167, 155, 145],
          [155, 145, 130],
          [ 98,  90,  73],
          ...,
          [155, 140, 133],
          [125, 111, 107],
          [101,  86,  83]],

         [[159, 149, 139],
          [123, 113, 100],
          [ 92,  83,  66],
          ...,
          [141, 127, 121],
          [127, 114, 108],
          [103,  90,  84]],

         ...,

         [[ 80,  62,  52],
          [ 82,  65,  55],
          [ 68,  53,  42],
          ...,
          [115, 101,  68],
          [125, 110,  80],
          [100,  84,  54]],

         [[ 81,  64,  54],
          [ 82,  65,  55],
          [ 62,  47,  38],
          ...,
          [122, 105,  69],
          [112,  94,  61],
          [143, 124,  89]],

         [[ 79,  61,  51],
          [ 79,  62,  52],
          [ 54, 

In [19]:
video_features_dict = diffusion_wrapper.extract_video_features(video_tensor, "")

In [20]:
video_features = diffusion_wrapper.concatenate_video_features({'up_block': video_features_dict['up_block'][0:3]})

In [21]:
video_features = video_features.permute(0, 2, 3, 1).float()
heatmaps = heatmap_generator.generate(video_features, (93, 137, 0))
tracks = zero_shot_tracker.track(heatmaps)

In [22]:
zero_shot_tracker.place_marker_in_frames(first_video['video'].squeeze(), tracks)
heatmap_generator.safe_heatmap_as_gif(heatmaps)